In [1]:
import sys
sys.path.append('/app/')  # noqa

from jina import Document, DocumentArray, Flow, Executor, requests
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import argparse
from memlish.executors.cache import RealSBERTEncoderCache
from memlish.executors.index import FaissIndexer
import torch
import hashlib

def my_hash(s): 
    return str(int(hashlib.md5(str(s).encode('utf-8')).hexdigest(), 16))

In [2]:
torch.multiprocessing.set_start_method('spawn', force=True) 

In [3]:
OUT_DIR = Path("/data/imgflip/v1/")
OUT_TEMPLATE_TEXTS_CSV = OUT_DIR/"template_texts.csv"

In [4]:
JINA_SBERT_EMBEDDING_TEMPLATE_TEXT_COLLECTION = "02_v1_images_sbert_all_mpnet_base_v2_imgflip_template_100k_embeddings"
MONGO_EMBEDDING_DB_NAME = 'memlish_db'

In [5]:
def text_input_docs(df):    
    for idx, row in df.iterrows():
        template_name = row["template_img_name"]
        try:
            template_text = row["text"]
            document_id = f"{template_name}_{my_hash(template_text)}"
            doc = Document(id=document_id, text=template_text)# tags={'filename': str(template_img_path)})
        except Exception as e:
            print(document_id)
            print(template_text)
            print(str(e))
        
        yield doc 

In [6]:
templates_df = pd.read_csv(OUT_TEMPLATE_TEXTS_CSV)

In [7]:
templates_df = templates_df[~templates_df['text'].isna()]

In [8]:
templates_df = templates_df[~templates_df.duplicated(["text", "template_img_name"])]

In [9]:
templates_df.head(2)

,Unnamed: 0,text,alt,views,upvotes,comments,template_name,template_img_name
0,0,WHERE IS THE CHOCKY MILK; THAT I SAVED LAST NI...,Chainsaw Bear | WHERE IS THE CHOCKY MILK; THA...,72,3,2,Chainsaw-Bear,Chainsaw-Bear.jpg
1,1,POV: YOU SAID POST OR VOID INSTEAD OF POINT OF...,Run | POV: YOU SAID POST OR VOID INSTEAD OF P...,69,2,0,Chainsaw-Bear,Chainsaw-Bear.jpg


In [10]:
k_to_take = 100
groups = []

for _, group in templates_df.groupby(['template_img_name']):
    selected_sorted = group.sort_values(by=['views'], ascending=False)[:k_to_take]
    groups.append(selected_sorted)

In [11]:
df = pd.concat(groups).reset_index(drop=True)
df.head(2)

,Unnamed: 0,text,alt,views,upvotes,comments,template_name,template_img_name
0,221950,"I WON $10,000,000 IN THE LOTTERY AND DECIDED T...","10 Guy | I WON $10,000,000 IN THE LOTTERY AND...",733523,305,107,10-Guy,10-Guy.jpg
1,228696,I NEED TO REFILL MY CHILD REPELLENT,10 Guy | I NEED TO REFILL MY CHILD REPELLENT |...,684545,12,0,10-Guy,10-Guy.jpg


In [13]:
embedder_params = {
        "device": 'cuda'
}

cache_params = {
    "embedder_params": embedder_params,
    "db_name": MONGO_EMBEDDING_DB_NAME, 
    "collection_name": JINA_SBERT_EMBEDDING_TEMPLATE_TEXT_COLLECTION,
    "embedding_field_name":'emb',
    "megabatch_size":8192
}

In [14]:
flow_text_encode = Flow().add(uses=RealSBERTEncoderCache, name="SBERT_encoder", uses_with=cache_params)
flow_text_encode

In [15]:
with flow_text_encode:
    flow_text_encode.post(on='', inputs=text_input_docs(df), request_size=10_000, show_progress=True)

⠹ 1/2 waiting SBERT_encoder to be ready...                                        

Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 1.08MB/s]


⠼ 1/2 waiting SBERT_encoder to be ready...                                        

Downloading: 100%|██████████| 10.1k/10.1k [00:00<00:00, 11.0MB/s]


⠧ 1/2 waiting SBERT_encoder to be ready...                                        

...

⠙ Working... ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:04:30  0.0 step/s 

  0%|          | 0/1 [00:00<?, ?it/s]

⠋ Working... ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:04:52  0.0 step/s {'es_tag': 'TIMELOG', 'timestamp': '2022-02-21 10:09:22.737511', 'function': 'memlish.executors.bert.get_embeddings', 'duration': 22.27311897277832}
⠋ Working... ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:04:55  0.0 step/s 

100%|██████████| 1/1 [00:25<00:00, 25.17s/it]


⠸       DONE ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:04:55  0.0 step/s 8 steps done in 4 minutes and 55 seconds
